In [195]:
import numpy as np
import pandas as pd
from sklearn import linear_model
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

In [214]:
X = pd.read_csv('./data/1_x.csv', header=None)
Y = pd.read_csv('./data/1_y.csv', header=None)

In [215]:
X = X.set_axis(['x1', 'x2', 'x3', 'x4', 'x5', 'x6', 'x7'], axis=1, copy=False)
Y = Y.set_axis(['y'], axis=1, copy=False)
pass

In [90]:
X

,x1,x2,x3,x4,x5,x6,x7
0,1.599865,0.340598,-0.603400,0.150586,-0.209677,0.955045,0.347682
1,1.472656,0.466832,-1.947130,0.625906,0.589354,0.753521,-2.138552
2,0.279378,-1.717121,0.251069,0.730013,0.301801,-0.206711,0.894811
3,0.450771,1.133196,-0.294028,-2.595338,0.313911,1.953527,0.359938
4,1.507042,0.932115,-0.789549,1.043712,-1.404000,-0.124354,-0.921117
...,...,...,...,...,...,...,...
120,1.018825,1.618920,-1.561572,1.057171,1.893181,-0.647510,1.220188
121,-1.244391,0.327816,-1.004231,-1.877654,1.307478,-1.839689,-0.176182
122,-1.968106,-0.054336,0.558046,-0.527741,-0.060449,0.531740,-0.374088
123,-0.970556,0.436830,0.877973,-0.970337,-0.282711,-1.464295,-0.976662


In [91]:
Y

,y
0,101.507602
1,-65.305680
2,152.189336
3,-109.702103
4,-11.188624
...,...
120,293.947002
121,-146.511266
122,-143.163481
123,-188.233401


In [186]:
print(f"количество точек данных: {X.__len__()}", f"количество признаков: {len(X.columns)}", sep="\n")
tab = "  "
display(X.describe())

количество точек данных: 125
количество признаков: 7


,x1,x2,x3,x4,x5,x6,x7
count,125.000000,125.000000,125.000000,125.000000,125.000000,125.000000,125.000000
mean,-0.046701,0.173034,0.049015,-0.005271,-0.095981,-0.107452,-0.056552
std,0.933281,0.928898,0.984631,1.000964,1.087336,0.928998,1.064615
min,-2.190258,-2.358885,-2.081062,-2.898741,-2.841371,-2.907825,-2.138552
25%,-0.775334,-0.429173,-0.657021,-0.685499,-0.869306,-0.672357,-0.936561
50%,0.030645,0.204892,-0.048617,0.054299,-0.102158,-0.174373,0.062705
75%,0.545665,0.755290,0.804735,0.659364,0.689661,0.531740,0.596198
max,2.092059,3.142216,2.272042,2.065050,2.660842,2.086050,3.026906


In [225]:
Xtr, Xt, Ytr, Yt = train_test_split(X, Y, test_size=0.2, random_state=0)

In [226]:
reg = linear_model.LinearRegression()

In [227]:
reg.fit(Xtr, Ytr)
pass

In [228]:
print(reg.intercept_)
print(reg.coef_)


[1.00692289]
[[50.92663567  0.51087097 23.67510908 69.78360243 56.39907333  0.2016185
  81.70528103]]


In [229]:
Yp = reg.predict(Xt)

In [266]:
df = pd.DataFrame({'Actual': np.reshape(Yt.to_numpy(), (1,-1))[0], 'Predicted': Yp.reshape(1,-1)[0]})
df

,Actual,Predicted
0,-131.233477,-119.476516
1,-22.452138,-24.526072
2,-212.860797,-199.505119
3,63.268545,64.037969
4,-170.781614,-177.859241
5,-254.290884,-242.259353
6,285.973829,271.183728
7,211.158999,214.640832
8,-198.420692,-192.406358
9,-274.437935,-263.466644


In [9]:
reg.score(X, Y)

0.9950364126599337

In [35]:
X

,x1,x2,x3,x4,x5,x6,x7
0,1.472656,0.466832,-1.947130,0.625906,0.589354,0.753521,-2.138552
1,0.279378,-1.717121,0.251069,0.730013,0.301801,-0.206711,0.894811
2,0.450771,1.133196,-0.294028,-2.595338,0.313911,1.953527,0.359938
3,1.507042,0.932115,-0.789549,1.043712,-1.404000,-0.124354,-0.921117
4,-0.793739,-1.249947,-0.048617,2.050793,-0.206069,-1.880998,0.596198
...,...,...,...,...,...,...,...
119,1.018825,1.618920,-1.561572,1.057171,1.893181,-0.647510,1.220188
120,-1.244391,0.327816,-1.004231,-1.877654,1.307478,-1.839689,-0.176182
121,-1.968106,-0.054336,0.558046,-0.527741,-0.060449,0.531740,-0.374088
122,-0.970556,0.436830,0.877973,-0.970337,-0.282711,-1.464295,-0.976662
